In [ ]:
using Plots
using Random
using LinearAlgebra
using ForwardDiff

using WeightedEnsemble
using TestLandscapes
using BasicMD
using JLD2

In [ ]:
Δb = 0.2
n_particles = 10^5;
seed = 100;
β = 10.0    # inverse temperature
b = 0.5 # target set [b, ∞)
x0 = [-1.0] # starting point

T = 4 # terminal time


Δt = 1e-2  # time step
nΔt = Int(T / Δt)
ΔT_recycle = 1e-2
nΔt_recycle = Int(ΔT_recycle / Δt) # number of time steps before applying recycler
nΔt_coarse = 1 * nΔt_recycle # number of time steps in a coarse step
@show n_we_steps = nΔt ÷ (nΔt_coarse);

x_voronoi = [[x - 0.5 * Δb] for x in -1.7:Δb:0.5+Δb]
n_bins = length(x_voronoi)

@show T;
@show Δt;
@show β;
@show n_particles;
@show Δb;
@show n_bins;
@show x_voronoi;

In [ ]:
V = x -> SymmetricDoubleWell(x[1]);
cfg = ForwardDiff.GradientConfig(V, zeros(Float64, 1));
∇V! = (gradV, X) -> ForwardDiff.gradient!(gradV, V, X, cfg);

fB = X -> Float64(X[1] > b);

sampler = EM(∇V!, β, Δt);

function restartA!(state::BasicMD.EMState)
    if (state.x[1] > b)
        @. state.x = x0
        ∇V!(state.∇V, x0)
    end
    state
end
constraints = Constraints(restartA!, trivial_constraint!, nΔt_recycle, nΔt_coarse);
mutation_opts = MDOptions(n_iters=nΔt_coarse, n_save_iters=nΔt_coarse);

mutation! = X -> sample_trajectory!(X, sampler, constraints, options=mutation_opts);

B0, bin_id, rebin! = setup_Voronoi_bins(x_voronoi);
E0 = Dirac_to_Ensemble(x0, n_particles)
rebin!(E0, B0, 0);


In [ ]:
B0.Ω

In [ ]:
data = jldopen("trivial_debug_309.jld2");
E = data["E"]
B = data["B"]

In [ ]:
@show sum(E.ω);
@show sum(B.ν);
@show B.ν[1:4];
@show B.n[1:4];

In [ ]:
histogram([x_[1] for x_ in E.ξ], label="Walkers")
title!("Unweighted Walker Distribution")
xlabel!("x")

In [ ]:
# E_ = deepcopy(E);
# B_ = deepcopy(B);
# E_, B_ = uniform_selection!(E_, B_, 0);

In [ ]:
sum(E.ω̂[findall(E.b̂ .==1)])

In [ ]:
E_ = deepcopy(E);
B_ = deepcopy(B);
rebin!(E_, B_, 0)
@. E_.o = 0;
@. B_.target = 0;
minimal_bin_allocation!(B_)
n_particles = length(E_)
# number of remaining particles to allocate
n_allocate = n_particles - sum(B_.target);
@show n_allocate;
uniform_bin_allocation!(B_, E_, n_allocate, 
    allocation_resampler = WeightedEnsemble.systematic);
within_bin_allocation!(E_, B_, within_bin_resampler = WeightedEnsemble.multinomial);
particle_ids = findall(isequal(1), E_.b);
@show length(E_.o[particle_ids]);
@show E_.o[particle_ids];


@show B_.target[1:4];
@show B_.ν[1];
@show B_.ν[1] / B_.target[1];
repopulate!(E_, B_);
copy!(E_.ω, E_.ω̂);
@. E_.ξ = deepcopy(E_.ξ̂);

rebin!(E_, B_, 0);


particle_ids = findall(isequal(1), E_.b);

# @show E_.ω̂[particle_ids[1:10]];
# @show B_.target[1:4];
 @show E_.ω[particle_ids[1:10]];
@show B_.ν[1];
# @xhow E.ω[particle_ids] / B.ν[p]

# ensure each nonempty bin has at least one particle
# WeightedEnsemble.minimal_bin_allocation!(B_)
# n_particles = length(E_)
# number of remaining particles to allocate
# n_allocate = n_particles - sum(B_.target)


In [ ]:
E_.ω[findall(E_.b .== 1)]

In [ ]:
B_.ν

In [ ]:
uniform_bin_allocation!(B_, E_, n_allocate, allocation_resampler = WeightedEnsemble.systematic)

In [ ]:
sum(B_.target)

In [ ]:
non_empty_bins = findall(n -> n > 0, B_.n)


In [ ]:
B_.ν

In [ ]:
within_bin_allocation!(E_, B_, within_bin_resampler = WeightedEnsemble.multinomial)
